In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [7]:
import sys  
from imp import reload

reload(sys)  
#sys.setdefaultencoding('utf8')

<module 'sys' (built-in)>

In [9]:
import sys
reload(sys)  
#sys.setdefaultencoding('Cp1252')

<module 'sys' (built-in)>

In [10]:
import mpld3
mpld3.enable_notebook()

In [11]:
import sys
sys.path.append('../../Utils/')

In [12]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10

In [13]:
import tensorflow as tf 
import numpy as np 
import pandas as pd 
import nltk
import math
import random
from sklearn.svm import SVC
from sklearn.utils import shuffle
from load_imdb_data import load_imdb_data
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np 
import matplotlib.pyplot as plt 
np.random.seed(0)

In [14]:
data = open("../../data/intent_data.txt").readlines()

In [15]:
data = data[1:]

In [16]:
sentence = []
labels = []

for item in data:
    temp = item.split('   ')
    sentence.append(temp[0].strip())
    labels.append(temp[1].strip().replace("\n", ''))

In [18]:
#sentence = [line.decode('utf-8').strip() for line in sentence]
sentence = [line.strip() for line in sentence]

In [19]:
sentence[:2], labels[:2]

(['I must look like a right weirdo .',
  "I really don't want to get up and get dressed for work"],
 ['non-intent', 'trifle'])

In [20]:
df = pd.DataFrame()
df['review'] = sentence
df['label'] = labels

In [21]:
df = df[df['label'].isin(['food', 'travel', 'career'])]

In [22]:
df.reset_index(drop=True, inplace=True)

In [23]:
df.head()

,review,label
0,I really want a hot dog . My co-workers went t...,food
1,"I wanna go to the beach , but ... Where is the...",travel
2,I really look forward to Egypt next week ! Its...,travel
3,I should buy more oatmeal cookies .. I just at...,food
4,"I want pancakes , should I make some ???",food


In [24]:
def processDocs(documents, vocab_size=5000):
    """
    This functions takes in a collection of documents and generates a vocabulary based on the size given in input. 
    It returns a representation for each document in the list of input documents. 
    """
    vocab = {} 
    doc_id = 0 
    doc_ids = []
    
    for doc in documents:
        doc_ids.append(doc_id)                          # Give an ID to each document 
        doc_id += 1
        
        for word in nltk.word_tokenize(doc):            # Generate a vocabulary while iterating threw the documents 
            if word not in vocab:
                vocab[word] = 1 
            else:
                vocab[word] += 1
    
    # Extract the most frequent words based on the vocabulary size 
    freq_words_list = sorted(vocab.items(), key=lambda x: x[1], reverse=True)[:vocab_size]
    freq_words_set = set([item[0] for item in freq_words_list])
    
    # Give an index to each word in vocabulary 
    word2idx = {}         
    index_word = 0
    for word in freq_words_set:
        word2idx[word] = index_word
        index_word += 1
    word2idx['UNK'] = index_word
    
    doc_repr = []                          # Represent each document with representation based on the vocabulary  
    for doc in documents:
        temp = []
        for w in doc:
            if w in word2idx:
                temp.append(word2idx[w])
            else:
                temp.append(word2idx['UNK'])
        doc_repr.append(temp)
        
    return documents, doc_ids, word2idx, doc_repr

In [25]:
docs, doc_ids, word2ids, doc_repr = processDocs(df['review'])

In [26]:
len(docs), len(doc_ids), len(word2ids), len(doc_repr)

(591, 591, 1901, 591)

## Architecture - PV-DM Distributed Memory version of Paragraph Vector

### Adding batchsizes for speedup

In [27]:
bucket_list = []

def generate_batch_pvdm(doc_ids, doc_repr, sample_size=10, batch_size=1000, window_size=7):
    global bucket_list

    docs_ids_to_select = list(set(doc_ids) - set(bucket_list))
    
    
    if len(docs_ids_to_select) < batch_size//sample_size:
        bucket_list = []
        docs_ids_to_select = doc_ids
        
    index = 0 
    train_wX = np.ndarray(shape=(batch_size, window_size), dtype=np.int32)
    train_dX = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    train_label = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    random_docs = random.sample(docs_ids_to_select, batch_size//sample_size)    # Choose set of random documents 

    bucket_list += random_docs
    
    for id_ in random_docs:
        for j in range(sample_size):                                 # Generating a dataset of sample size 
            random_index = random.randint(0, len(doc_repr[id_]) - window_size - 1)
            sample_window = doc_repr[id_][random_index: random_index + window_size + 1]
            train_wX[index] = sample_window[:-1]
            train_dX[index] = id_
            train_label[index] = sample_window[-1]  
            index += 1
    return train_wX, train_dX, train_label 

In [28]:
doc_size = len(docs)
embedding_size_w = 100
embedding_size_d = 100
vocab_size = len(word2ids)
window_size = 7
n_neg_samples = 30
learning_rate = 10e-6
epochs = 10001
batch_size=1000
mu=0.9
combined_embed_vector_length = embedding_size_d + embedding_size_w

In [29]:
# Define placeholders for training 
train_wX = tf.placeholder(tf.int32, shape=[batch_size, window_size])
train_dX = tf.placeholder(tf.int32, shape=[batch_size, 1])
train_label = tf.placeholder(tf.int32, shape=[batch_size, 1])

In [30]:
doc_embedding_np = np.random.randn(doc_size, embedding_size_d)/np.sqrt(doc_size + embedding_size_d)
word_embedding_np = np.random.randn(vocab_size, embedding_size_w)/np.sqrt(vocab_size + embedding_size_w)

In [31]:
# Define matrix for doc_embedding and word_embedding 
doc_embedding = tf.Variable(doc_embedding_np.astype(np.float32), name="doc_embedding")
word_embedding = tf.Variable(word_embedding_np.astype(np.float32),name="word_embedding")

In [32]:
# Define weights for the output unit 
weights = tf.Variable(tf.truncated_normal([vocab_size, combined_embed_vector_length], 
                                       stddev=1.0 / math.sqrt(combined_embed_vector_length)))
biases = tf.Variable(tf.zeros(vocab_size))

In [34]:
embed = []

# generating a vector of size embedding_size_d
embed_w = tf.zeros([1, embedding_size_w], dtype=tf.float32)

# add all the word vecs in window_size
for j in range(window_size):
    embed_w += tf.nn.embedding_lookup(word_embedding, train_wX[:, j])
embed.append(embed_w)

# Add the doc2vec from the doc_embedding 
embed_d = tf.nn.embedding_lookup(doc_embedding, train_dX[:, 0])
embed.append(embed_d)

print (embed_w, embed_d)

embed = tf.concat(embed, 1)

Tensor("add_6:0", shape=(1000, 100), dtype=float32) Tensor("embedding_lookup_7:0", shape=(1000, 100), dtype=float32)


In [35]:
loss = tf.nn.sampled_softmax_loss(weights=weights, \
                                  biases=biases, \
                                  labels=train_label, \
                                  inputs=embed, \
                                  num_sampled=n_neg_samples, \
                                  num_classes=vocab_size)

In [36]:
loss = tf.reduce_mean(loss)

In [37]:
optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)

In [38]:
import os
os.system("mkdir /dev/shm/tensorflow_models")

256

In [40]:
saver = tf.train.Saver()

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    average_loss = 0
    
    for step in range(epochs):
        epoch_error = 0.0
        temp_wX , temp_dX, temp_labels = generate_batch_pvdm(doc_ids=doc_ids, doc_repr=doc_repr)
        feed_dict = {train_wX : temp_wX, train_dX : temp_dX,train_label : temp_labels}
        op, l = sess.run([optimizer, loss], 
                                    feed_dict=feed_dict)
        
        epoch_error += l
                
        if step % 100 == 0:
            print ("Error at epoch : ", step, " = ", epoch_error)
            
    save_path = saver.save(sess, "/dev/shm/tensorflow_models/model_pvdm_batch_training.ckpt")
    print("Model saved in file: %s" % save_path)

Error at epoch :  0  =  2.39558482170105
Error at epoch :  100  =  2.1010963916778564
Error at epoch :  200  =  2.4926207065582275
Error at epoch :  300  =  1.9733412265777588
Error at epoch :  400  =  2.568958282470703
Error at epoch :  500  =  2.240765333175659
Error at epoch :  600  =  2.350950002670288
Error at epoch :  700  =  2.4614598751068115
Error at epoch :  800  =  2.0558061599731445
Error at epoch :  900  =  2.5341153144836426
Error at epoch :  1000  =  2.0842864513397217
Error at epoch :  1100  =  2.173076629638672
Error at epoch :  1200  =  1.8386852741241455
Error at epoch :  1300  =  2.1438727378845215
Error at epoch :  1400  =  1.8991338014602661
Error at epoch :  1500  =  1.9410735368728638
Error at epoch :  1600  =  2.2806317806243896
Error at epoch :  1700  =  1.8958332538604736
Error at epoch :  1800  =  2.253342390060425
Error at epoch :  1900  =  1.7826446294784546
Error at epoch :  2000  =  1.7984576225280762
Error at epoch :  2100  =  2.1114745140075684
Error a

### Evaluation of the representation 

In [41]:
doc_pvdm = None

with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, "/dev/shm/tensorflow_models/model_pvdm_batch_training.ckpt")
    print("Model restored.")
    doc2vec = doc_pvdm = doc_embedding.eval()
    #performanceTest(doc2vec, list(imdb_data['sentiment']), method=None)

INFO:tensorflow:Restoring parameters from /dev/shm/tensorflow_models/model_pvdm_batch_training.ckpt
Model restored.


In [42]:
doc_pvdm

array([[ 0.06710316,  0.01522323,  0.037236  , ...,  0.06792206,
         0.00482333,  0.01528981],
       [ 0.0716304 , -0.05127098, -0.04832826, ...,  0.03131485,
         0.08228817,  0.05084361],
       [-0.01405153, -0.00910595,  0.041837  , ...,  0.02216267,
        -0.01519922,  0.01407307],
       ...,
       [-0.05926265, -0.02806516, -0.04147395, ...,  0.04507519,
        -0.03191489,  0.02484348],
       [-0.02171955,  0.00578829, -0.02006715, ..., -0.03002608,
         0.00851845,  0.00016619],
       [ 0.0642072 , -0.00572014, -0.01003487, ...,  0.04401367,
        -0.02242042, -0.04742105]], dtype=float32)

In [43]:
from sklearn.manifold import TSNE
model = TSNE(perplexity=50, n_iter=5000)
Z = model.fit_transform(doc_pvdm) 

In [44]:
df['axis1'] = Z[:, 0]
df['axis2'] = Z[:, 1]

In [45]:
df.reset_index(drop=True, inplace=True)

In [46]:
df.head()

,review,label,axis1,axis2
0,I really want a hot dog . My co-workers went t...,food,-1.444408,0.317180
1,"I wanna go to the beach , but ... Where is the...",travel,0.022460,-1.047988
2,I really look forward to Egypt next week ! Its...,travel,-1.654806,-0.251050
3,I should buy more oatmeal cookies .. I just at...,food,-0.335669,1.979056
4,"I want pancakes , should I make some ???",food,0.564902,0.377289


In [47]:
from ggplot import *
ggplot(aes(x='axis1', y='axis2', color='label'), data=df)  + geom_point()

/home/piyush/Piyush/study/anthill/anthill/lib/python3.5/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/home/piyush/Piyush/study/anthill/anthill/lib/python3.5/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp


ImportError: cannot import name 'Timestamp'